# **Directed Weighted Graphs - Complete Guide**

A **Directed Weighted Graph** (also called a **Weighted Digraph**) is a collection of vertices (nodes) connected by directed edges where each edge has both a specific direction and an associated weight (cost, distance, capacity, etc.). This combines the directional constraints of directed graphs with the quantitative relationships of weighted graphs.

**Key Features:**
- **Vertices (Nodes)**: Individual points or entities in the graph
- **Directed Weighted Edges**: Connections with specific direction and numerical weight (u → v, weight w)
- **Asymmetric Relations**: Edge (u,v) with weight w₁ does NOT imply edge (v,u) exists or has weight w₁
- **Edge Weights**: Represent costs, distances, capacities, probabilities, or any measurable relationship
- **Path Weight**: Sum of all edge weights along a path
- **Shortest Path**: Path with minimum total weight (not necessarily minimum hops)

**Graph Terminology:**
- **Weighted In-degree**: Sum of weights of incoming edges
- **Weighted Out-degree**: Sum of weights of outgoing edges
- **Shortest Path Distance**: Minimum weight path between two vertices
- **Negative Edge Weights**: Can create negative cycles and complicate shortest path algorithms
- **All-Pairs Shortest Paths**: Shortest distances between every pair of vertices
- **Minimum Spanning Arborescence**: Directed version of MST (rooted at a specific vertex)

**Mathematical Representation:**
- **Graph G = (V, E, W)** where V is vertices, E is directed edges, W is weight function
- **Weight Function**: W: E → ℝ (maps each edge to a real number)
- **Path Weight**: w(p) = Σ w(eᵢ) for all edges eᵢ in path p
- **Distance**: δ(u,v) = min{w(p) : p is a path from u to v}

**Applications:**
- **Transportation Networks**: Flight routes with costs, road networks with travel times
- **Computer Networks**: Routing protocols with latency/bandwidth considerations
- **Project Management**: Task dependencies with durations and costs
- **Financial Systems**: Currency exchange rates, transaction costs
- **Social Networks**: Influence strength, relationship weights
- **Supply Chain**: Logistics costs, delivery times, capacity constraints
- **Game Theory**: Decision trees with payoffs, strategy costs
- **Resource Allocation**: Optimization problems with weighted constraints

**Weight Interpretation Examples:**
- **Distance**: GPS navigation, shortest route planning
- **Time**: Scheduling algorithms, critical path analysis
- **Cost**: Economic optimization, budget allocation
- **Capacity**: Network flow, resource management
- **Probability**: Markov chains, probabilistic models
- **Energy**: Power grid optimization, circuit analysis
- **Bandwidth**: Network routing, data transmission
- **Risk**: Financial modeling, decision analysis

## Node Definition for Directed Weighted Graphs

A **Node (Vertex)** in a directed weighted graph represents an entity with directional relationships. Each node can have different incoming and outgoing connections with associated weights.

In directed weighted graphs, we need to store:
- **Node Identity**: Unique identifier (name, number, etc.)
- **Outgoing Edges**: Nodes this vertex points to with weights
- **Incoming Edges**: Nodes that point to this vertex with weights  
- **Traversal State**: Visited status, distances, parent pointers (for algorithms)

We'll represent nodes as simple identifiers and manage their directed connections through graph data structures that store both adjacency and weight information while respecting edge directions.

In [16]:
# Import all required libraries
import random
from collections import defaultdict, deque
import heapq
import sys
import math

# Create a sample directed weighted graph with 7 nodes
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Define directed edges with weights for our sample graph
# Format: (from_node, to_node, weight) - direction matters!
edges = [
    ('A', 'B', 4), ('A', 'C', 2),
    ('B', 'D', 5), ('B', 'E', 1),
    ('C', 'D', 8), ('C', 'F', 10),
    ('D', 'E', 2), ('D', 'G', 6),
    ('E', 'F', 3), ('E', 'G', 1),
    ('F', 'G', 4),
    ('G', 'A', 7),  # Creates a cycle back to A
    ('C', 'B', 3)   # Alternative path from C to B
]

print("=== Sample Directed Weighted Graph ===")
print(f"Nodes: {nodes}")
print(f"\nDirected edges with weights:")
for u, v, w in edges:
    print(f"  {u} → {v} (weight: {w})")

=== Sample Directed Weighted Graph ===
Nodes: ['A', 'B', 'C', 'D', 'E', 'F', 'G']

Directed edges with weights:
  A → B (weight: 4)
  A → C (weight: 2)
  B → D (weight: 5)
  B → E (weight: 1)
  C → D (weight: 8)
  C → F (weight: 10)
  D → E (weight: 2)
  D → G (weight: 6)
  E → F (weight: 3)
  E → G (weight: 1)
  F → G (weight: 4)
  G → A (weight: 7)
  C → B (weight: 3)


In [17]:
class DirectedWeightedGraph:
    def __init__(self, nodes):
        self.nodes = nodes
        self.node_to_index = {node: i for i, node in enumerate(nodes)}
        self.num_nodes = len(nodes)
        
        # Adjacency Matrix representation
        self.adj_matrix = [[0 if i == j else float('inf') for j in range(self.num_nodes)] 
                          for i in range(self.num_nodes)]
        
        # Adjacency List representation (outgoing edges)
        self.adj_list = defaultdict(list)
        
        # Reverse adjacency list (incoming edges) for some algorithms
        self.reverse_adj_list = defaultdict(list)
    
    def add_edge(self, u, v, weight):
        """Add directed weighted edge from vertex u to vertex v"""
        u_idx = self.node_to_index[u]
        v_idx = self.node_to_index[v]
        
        # Update adjacency matrix
        self.adj_matrix[u_idx][v_idx] = weight
        
        # Update adjacency lists
        self.adj_list[u].append((v, weight))
        self.reverse_adj_list[v].append((u, weight))
    
    def display_matrix(self):
        """Display adjacency matrix representation"""
        print("Adjacency Matrix (rows=from, cols=to):")
        print("    ", end="")
        for node in self.nodes:
            print(f"{node:>4}", end="")
        print()
        
        for i, node in enumerate(self.nodes):
            print(f"{node:>2}: ", end="")
            for j in range(self.num_nodes):
                val = self.adj_matrix[i][j]
                if val == float('inf'):
                    print("  ∞", end="")
                else:
                    print(f"{val:>3}", end="")
            print()
    
    def display_list(self):
        """Display adjacency list representation"""
        print("Adjacency List (outgoing edges):")
        for node in self.nodes:
            neighbors = [(neighbor, weight) for neighbor, weight in self.adj_list[node]]
            print(f"{node}: {neighbors}")
        
        print("\nReverse Adjacency List (incoming edges):")
        for node in self.nodes:
            predecessors = [(pred, weight) for pred, weight in self.reverse_adj_list[node]]
            print(f"{node}: {predecessors}")
    
    def get_in_degree(self, node):
        """Get in-degree (number of incoming edges) of a node"""
        return len(self.reverse_adj_list[node])
    
    def get_out_degree(self, node):
        """Get out-degree (number of outgoing edges) of a node"""
        return len(self.adj_list[node])
    
    def get_weighted_in_degree(self, node):
        """Get sum of weights of incoming edges"""
        return sum(weight for _, weight in self.reverse_adj_list[node])
    
    def get_weighted_out_degree(self, node):
        """Get sum of weights of outgoing edges"""
        return sum(weight for _, weight in self.adj_list[node])

# Create and populate our sample directed graph
graph = DirectedWeightedGraph(nodes)

# Add all directed edges to both representations
for u, v, weight in edges:
    graph.add_edge(u, v, weight)

print("=== Graph Representations ===")
graph.display_matrix()
print()
graph.display_list()

print("\n=== Degree Analysis ===")
for node in nodes:
    in_deg = graph.get_in_degree(node)
    out_deg = graph.get_out_degree(node)
    weighted_in = graph.get_weighted_in_degree(node)
    weighted_out = graph.get_weighted_out_degree(node)
    print(f"{node}: in-degree={in_deg}, out-degree={out_deg}, weighted-in={weighted_in}, weighted-out={weighted_out}")

=== Graph Representations ===
Adjacency Matrix (rows=from, cols=to):
       A   B   C   D   E   F   G
 A:   0  4  2  ∞  ∞  ∞  ∞
 B:   ∞  0  ∞  5  1  ∞  ∞
 C:   ∞  3  0  8  ∞ 10  ∞
 D:   ∞  ∞  ∞  0  2  ∞  6
 E:   ∞  ∞  ∞  ∞  0  3  1
 F:   ∞  ∞  ∞  ∞  ∞  0  4
 G:   7  ∞  ∞  ∞  ∞  ∞  0

Adjacency List (outgoing edges):
A: [('B', 4), ('C', 2)]
B: [('D', 5), ('E', 1)]
C: [('D', 8), ('F', 10), ('B', 3)]
D: [('E', 2), ('G', 6)]
E: [('F', 3), ('G', 1)]
F: [('G', 4)]
G: [('A', 7)]

Reverse Adjacency List (incoming edges):
A: [('G', 7)]
B: [('A', 4), ('C', 3)]
C: [('A', 2)]
D: [('B', 5), ('C', 8)]
E: [('B', 1), ('D', 2)]
F: [('C', 10), ('E', 3)]
G: [('D', 6), ('E', 1), ('F', 4)]

=== Degree Analysis ===
A: in-degree=1, out-degree=2, weighted-in=7, weighted-out=6
B: in-degree=2, out-degree=2, weighted-in=7, weighted-out=6
C: in-degree=1, out-degree=3, weighted-in=2, weighted-out=21
D: in-degree=2, out-degree=2, weighted-in=13, weighted-out=8
E: in-degree=2, out-degree=2, weighted-in=3, weighted-o

## Breadth-First Search (BFS) on Directed Graphs

**BFS** in directed graphs follows outgoing edges only, exploring nodes level by level from a starting vertex. It finds the **shortest path in terms of hops** while respecting edge directions (ignoring weights).

**Algorithm Steps:**
1. Start from source vertex, mark it visited
2. Add source to queue  
3. While queue is not empty:
   - Dequeue vertex
   - Visit all unvisited successors (via outgoing edges)
   - Mark successors as visited and enqueue them
4. Process vertices in the order they are dequeued

**Time Complexity:** O(V + E) where V is vertices and E is directed edges
**Space Complexity:** O(V) for the queue and visited set

**Use Cases:**
- **Reachability analysis** from a source node
- **Shortest hop distance** in directed networks
- **Level-order traversal** of directed graphs

In [18]:
def bfs_directed(graph, start_node):
    """
    Breadth-First Search traversal of directed graph
    Returns BFS order, distances, and parent pointers
    """
    visited = set()
    queue = deque([start_node])
    bfs_order = []
    distances = {start_node: 0}
    parent = {start_node: None}
    
    visited.add(start_node)
    
    print(f"Starting BFS from node: {start_node}")
    print("Step by step traversal (following edge directions):")
    
    step = 1
    while queue:
        current_node = queue.popleft()
        bfs_order.append(current_node)
        
        print(f"Step {step}: Visit {current_node} (distance: {distances[current_node]})")
        
        # Get successors from adjacency list (outgoing edges only)
        successors = []
        for neighbor, weight in graph.adj_list[current_node]:
            if neighbor not in visited:
                successors.append((neighbor, weight))
                visited.add(neighbor)
                queue.append(neighbor)
                distances[neighbor] = distances[current_node] + 1
                parent[neighbor] = current_node
        
        if successors:
            successor_names = [f"{n}(w:{w})" for n, w in successors]
            print(f"         Add to queue: {successor_names}")
        
        print(f"         Queue state: {list(queue)}")
        step += 1
        print()
    
    # Check reachability
    reachable = set(bfs_order)
    unreachable = set(graph.nodes) - reachable
    
    return bfs_order, distances, parent, reachable, unreachable

# Perform BFS traversal
bfs_result, bfs_distances, bfs_parent, reachable, unreachable = bfs_directed(graph, 'A')

print("=== BFS Results ===")
print(f"BFS Order: {' → '.join(bfs_result)}")
print(f"Distances from A: {bfs_distances}")
print(f"Parent nodes: {bfs_parent}")
print(f"Reachable from A: {sorted(reachable)}")
if unreachable:
    print(f"Unreachable from A: {sorted(unreachable)}")
else:
    print("All nodes reachable from A")

Starting BFS from node: A
Step by step traversal (following edge directions):
Step 1: Visit A (distance: 0)
         Add to queue: ['B(w:4)', 'C(w:2)']
         Queue state: ['B', 'C']

Step 2: Visit B (distance: 1)
         Add to queue: ['D(w:5)', 'E(w:1)']
         Queue state: ['C', 'D', 'E']

Step 3: Visit C (distance: 1)
         Add to queue: ['F(w:10)']
         Queue state: ['D', 'E', 'F']

Step 4: Visit D (distance: 2)
         Add to queue: ['G(w:6)']
         Queue state: ['E', 'F', 'G']

Step 5: Visit E (distance: 2)
         Queue state: ['F', 'G']

Step 6: Visit F (distance: 2)
         Queue state: ['G']

Step 7: Visit G (distance: 3)
         Queue state: []

=== BFS Results ===
BFS Order: A → B → C → D → E → F → G
Distances from A: {'A': 0, 'B': 1, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 3}
Parent nodes: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B', 'E': 'B', 'F': 'C', 'G': 'D'}
Reachable from A: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
All nodes reachable from A


## Depth-First Search (DFS) - Preorder, Postorder

**DFS** in directed graphs explores as far as possible along each directed path before backtracking. For directed graphs, we focus on:

### 1. Preorder DFS
- **Process vertex** before visiting its successors
- **Good for**: Node discovery, path exploration

### 2. Postorder DFS
- **Process vertex** after visiting all its successors
- **Good for**: Topological sorting, finish time tracking

**Algorithm Steps:**
1. Mark current vertex as visited
2. For Preorder: Process current vertex
3. Recursively visit all unvisited successors (via outgoing edges)
4. For Postorder: Process current vertex after successors

**Time Complexity:** O(V + E) 
**Space Complexity:** O(V) for recursion stack and visited array

**Use Cases:**
- **Cycle detection** in directed graphs
- **Topological sorting** (Postorder)
- **Strongly connected components**
- **Path finding** with backtracking

In [19]:
def dfs_preorder_directed(graph, start_node, visited=None, order=None, step_counter=None):
    """DFS with Preorder traversal (process node before successors)"""
    if visited is None:
        visited = set()
        order = []
        step_counter = [1]
        print(f"Starting DFS Preorder from node: {start_node}")
        print("Step by step traversal (following edge directions):")
    
    # Preorder: Process current node first
    visited.add(start_node)
    order.append(start_node)
    
    print(f"Step {step_counter[0]}: Process {start_node} (Preorder)")
    step_counter[0] += 1
    
    # Visit all unvisited successors (outgoing edges)
    successors = [neighbor for neighbor, weight in graph.adj_list[start_node]]
    for successor in sorted(successors):  # Sort for consistent output
        if successor not in visited:
            print(f"         Recursively visit {successor}")
            dfs_preorder_directed(graph, successor, visited, order, step_counter)
    
    return order

def dfs_postorder_directed(graph, start_node, visited=None, order=None, step_counter=None):
    """DFS with Postorder traversal (process node after successors)"""
    if visited is None:
        visited = set()
        order = []
        step_counter = [1]
        print(f"Starting DFS Postorder from node: {start_node}")
        print("Step by step traversal (following edge directions):")
    
    visited.add(start_node)
    
    # Visit all unvisited successors first (outgoing edges)
    successors = [neighbor for neighbor, weight in graph.adj_list[start_node]]
    for successor in sorted(successors):
        if successor not in visited:
            print(f"Step {step_counter[0]}: Recursively visit {successor}")
            step_counter[0] += 1
            dfs_postorder_directed(graph, successor, visited, order, step_counter)
    
    # Postorder: Process current node after all successors
    order.append(start_node)
    print(f"Step {step_counter[0]}: Process {start_node} (Postorder)")
    step_counter[0] += 1
    
    return order

# Perform DFS variations
print("=== DFS Preorder ===")
preorder_result = dfs_preorder_directed(graph, 'A')
print(f"Preorder: {' → '.join(preorder_result)}")
print()

print("=== DFS Postorder ===")
postorder_result = dfs_postorder_directed(graph, 'A')
print(f"Postorder: {' → '.join(postorder_result)}")
print()

print("=== Comparison with BFS ===")
print(f"BFS Order:       {' → '.join(bfs_result)}")
print(f"DFS Preorder:    {' → '.join(preorder_result)}")
print(f"DFS Postorder:   {' → '.join(postorder_result)}")

=== DFS Preorder ===
Starting DFS Preorder from node: A
Step by step traversal (following edge directions):
Step 1: Process A (Preorder)
         Recursively visit B
Step 2: Process B (Preorder)
         Recursively visit D
Step 3: Process D (Preorder)
         Recursively visit E
Step 4: Process E (Preorder)
         Recursively visit F
Step 5: Process F (Preorder)
         Recursively visit G
Step 6: Process G (Preorder)
         Recursively visit C
Step 7: Process C (Preorder)
Preorder: A → B → D → E → F → G → C

=== DFS Postorder ===
Starting DFS Postorder from node: A
Step by step traversal (following edge directions):
Step 1: Recursively visit B
Step 2: Recursively visit D
Step 3: Recursively visit E
Step 4: Recursively visit F
Step 5: Recursively visit G
Step 6: Process G (Postorder)
Step 7: Process F (Postorder)
Step 8: Process E (Postorder)
Step 9: Process D (Postorder)
Step 10: Process B (Postorder)
Step 11: Recursively visit C
Step 12: Process C (Postorder)
Step 13: Process 

## Dijkstra's Algorithm - Shortest Path in Directed Graphs

**Dijkstra's Algorithm** finds the shortest paths from a source vertex to all other vertices in a directed weighted graph with **non-negative edge weights**. It respects edge directions.

**Algorithm Steps:**
1. Initialize distances to all vertices as infinity, except source (distance 0)
2. Create a min-heap with all vertices
3. While heap is not empty:
   - Extract vertex with minimum distance
   - For each successor (via outgoing edges), if distance through current vertex is shorter, update
   - Add updated vertices back to heap
4. Result: shortest distances and paths to all reachable vertices

**Time Complexity:** O((V + E) log V) with binary heap
**Space Complexity:** O(V) for distance and parent arrays

**Key Properties:**
- **Greedy approach**: Always picks the closest unvisited vertex
- **Directional**: Only follows outgoing edges
- **Non-negative weights**: Cannot handle negative edge weights

**Use Cases:**
- **GPS Navigation**: Finding shortest routes in road networks
- **Network Routing**: Internet packet routing protocols
- **Game AI**: Pathfinding with directional movement costs

In [20]:
def dijkstra_directed(graph, start_node):
    """
    Dijkstra's algorithm for finding shortest paths in directed graph
    Returns distances and parent pointers for path reconstruction
    """
    # Initialize distances and parent pointers
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    parent = {node: None for node in graph.nodes}
    visited = set()
    
    # Priority queue: (distance, node)
    pq = [(0, start_node)]
    
    print(f"Starting Dijkstra's algorithm from node: {start_node}")
    print("Step by step execution:")
    
    step = 1
    while pq:
        current_dist, current_node = heapq.heappop(pq)
        
        # Skip if already processed
        if current_node in visited:
            continue
            
        visited.add(current_node)
        
        print(f"Step {step}: Process {current_node} (distance: {current_dist})")
        
        # Check all successors (outgoing edges only)
        updates = []
        for successor, weight in graph.adj_list[current_node]:
            if successor not in visited:
                new_distance = distances[current_node] + weight
                
                if new_distance < distances[successor]:
                    distances[successor] = new_distance
                    parent[successor] = current_node
                    heapq.heappush(pq, (new_distance, successor))
                    updates.append(f"{successor}: {new_distance}")
        
        if updates:
            print(f"         Updated distances: {', '.join(updates)}")
        
        # Show current priority queue state
        pq_state = [(dist, node) for dist, node in pq if node not in visited]
        if pq_state:
            print(f"         Priority queue: {pq_state}")
        
        step += 1
        print()
    
    return distances, parent

def reconstruct_path(parent, start, end):
    """Reconstruct shortest path from start to end using parent pointers"""
    if parent[end] is None and end != start:
        return []
    
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = parent[current]
    
    return path[::-1]  # Reverse to get path from start to end

# Run Dijkstra's algorithm
dijkstra_distances, dijkstra_parent = dijkstra_directed(graph, 'A')

print("=== Dijkstra's Results ===")
print(f"Shortest distances from A: {dijkstra_distances}")
print(f"Parent pointers: {dijkstra_parent}")

print("\nShortest paths from A to all nodes:")
for node in graph.nodes:
    if node != 'A':
        path = reconstruct_path(dijkstra_parent, 'A', node)
        if path:
            path_weight = dijkstra_distances[node]
            print(f"A → {node}: {' → '.join(path)} (weight: {path_weight})")
        else:
            print(f"A → {node}: No path exists")

Starting Dijkstra's algorithm from node: A
Step by step execution:
Step 1: Process A (distance: 0)
         Updated distances: B: 4, C: 2
         Priority queue: [(2, 'C'), (4, 'B')]

Step 2: Process C (distance: 2)
         Updated distances: D: 10, F: 12
         Priority queue: [(4, 'B'), (10, 'D'), (12, 'F')]

Step 3: Process B (distance: 4)
         Updated distances: D: 9, E: 5
         Priority queue: [(5, 'E'), (9, 'D'), (10, 'D'), (12, 'F')]

Step 4: Process E (distance: 5)
         Updated distances: F: 8, G: 6
         Priority queue: [(6, 'G'), (8, 'F'), (10, 'D'), (12, 'F'), (9, 'D')]

Step 5: Process G (distance: 6)
         Priority queue: [(8, 'F'), (9, 'D'), (10, 'D'), (12, 'F')]

Step 6: Process F (distance: 8)
         Priority queue: [(9, 'D'), (10, 'D')]

Step 7: Process D (distance: 9)

=== Dijkstra's Results ===
Shortest distances from A: {'A': 0, 'B': 4, 'C': 2, 'D': 9, 'E': 5, 'F': 8, 'G': 6}
Parent pointers: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B', 'E': 'B',

## Bellman-Ford Algorithm - Handles Negative Weights

**Bellman-Ford Algorithm** finds shortest paths from a source vertex to all other vertices in a directed graph, and can handle **negative edge weights**. It also **detects negative cycles**.

**Algorithm Steps:**
1. Initialize distances to all vertices as infinity, except source (distance 0)
2. For (V-1) iterations:
   - For each directed edge (u,v) with weight w:
     - If distance[u] + w < distance[v], update distance[v] and parent[v]
3. Run one more iteration to detect negative cycles
4. If any distance can still be updated, negative cycle exists

**Time Complexity:** O(V × E) - always, regardless of graph structure
**Space Complexity:** O(V) for distance and parent arrays

**Key Properties:**
- **Dynamic Programming**: Builds up shortest paths iteratively
- **Negative weight handling**: Can process negative edge weights
- **Negative cycle detection**: Identifies if shortest paths are undefined
- **Directional**: Respects edge directions

**Use Cases:**
- **Currency arbitrage**: Detecting profitable exchange cycles
- **Network analysis**: Routing with quality metrics (can be negative)
- **Game theory**: Finding optimal strategies with penalties

In [21]:
def bellman_ford_directed(graph, start_node):
    """
    Bellman-Ford algorithm for directed graphs with negative weight support
    Returns distances, parent pointers, and whether negative cycle exists
    """
    # Initialize distances and parent pointers
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    parent = {node: None for node in graph.nodes}
    
    # Convert adjacency list to edge list for easier processing
    edge_list = []
    for u in graph.adj_list:
        for v, weight in graph.adj_list[u]:
            edge_list.append((u, v, weight))
    
    print(f"Starting Bellman-Ford algorithm from node: {start_node}")
    print(f"Directed edge list: {edge_list}")
    print("Step by step execution:")
    
    # Relax edges V-1 times
    for iteration in range(len(graph.nodes) - 1):
        print(f"\nIteration {iteration + 1}:")
        updated = False
        
        for u, v, weight in edge_list:
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                old_distance = distances[v]
                distances[v] = distances[u] + weight
                parent[v] = u
                updated = True
                print(f"  Updated {u}→{v}: {old_distance} → {distances[v]} (edge weight: {weight})")
        
        if not updated:
            print(f"  No updates in iteration {iteration + 1}, algorithm can terminate early")
            break
        
        print(f"  Current distances: {distances}")
    
    # Check for negative cycles
    print(f"\nNegative cycle detection:")
    has_negative_cycle = False
    negative_cycle_edges = []
    
    for u, v, weight in edge_list:
        if distances[u] != float('inf') and distances[u] + weight < distances[v]:
            has_negative_cycle = True
            negative_cycle_edges.append((u, v, weight))
            print(f"  Negative cycle detected involving edge {u} → {v} (weight: {weight})")
    
    if not has_negative_cycle:
        print(f"  No negative cycle detected")
    
    return distances, parent, has_negative_cycle

# Run Bellman-Ford algorithm on our graph
bf_distances, bf_parent, has_neg_cycle = bellman_ford_directed(graph, 'A')

print("\n=== Bellman-Ford Results ===")
print(f"Shortest distances from A: {bf_distances}")
print(f"Parent pointers: {bf_parent}")
print(f"Negative cycle exists: {has_neg_cycle}")

print("\nShortest paths from A to all nodes:")
for node in graph.nodes:
    if node != 'A':
        if not has_neg_cycle and bf_distances[node] != float('inf'):
            path = reconstruct_path(bf_parent, 'A', node)
            if path:
                path_weight = bf_distances[node]
                print(f"A → {node}: {' → '.join(path)} (weight: {path_weight})")
        elif has_neg_cycle:
            print(f"A → {node}: Undefined due to negative cycle")
        else:
            print(f"A → {node}: No path exists")

# Demonstrate with a graph that has negative weights
print("\n=== Testing with Negative Weights ===")
negative_edges = [
    ('A', 'B', 4), ('A', 'C', 2),
    ('B', 'C', -6), ('B', 'D', 5),  # Negative weight edge
    ('C', 'D', 1), ('C', 'A', -1)   # Creates negative cycle: A→C→A
]

neg_graph = DirectedWeightedGraph(['A', 'B', 'C', 'D'])
for u, v, w in negative_edges:
    neg_graph.add_edge(u, v, w)

print("\nGraph with negative weights and potential negative cycle:")
for u, v, w in negative_edges:
    print(f"  {u} → {v} (weight: {w})")

neg_distances, neg_parent, neg_cycle = bellman_ford_directed(neg_graph, 'A')
print(f"\nResults: Distances = {neg_distances}, Has negative cycle = {neg_cycle}")

Starting Bellman-Ford algorithm from node: A
Directed edge list: [('A', 'B', 4), ('A', 'C', 2), ('B', 'D', 5), ('B', 'E', 1), ('C', 'D', 8), ('C', 'F', 10), ('C', 'B', 3), ('D', 'E', 2), ('D', 'G', 6), ('E', 'F', 3), ('E', 'G', 1), ('F', 'G', 4), ('G', 'A', 7)]
Step by step execution:

Iteration 1:
  Updated A→B: inf → 4 (edge weight: 4)
  Updated A→C: inf → 2 (edge weight: 2)
  Updated B→D: inf → 9 (edge weight: 5)
  Updated B→E: inf → 5 (edge weight: 1)
  Updated C→F: inf → 12 (edge weight: 10)
  Updated D→G: inf → 15 (edge weight: 6)
  Updated E→F: 12 → 8 (edge weight: 3)
  Updated E→G: 15 → 6 (edge weight: 1)
  Current distances: {'A': 0, 'B': 4, 'C': 2, 'D': 9, 'E': 5, 'F': 8, 'G': 6}

Iteration 2:
  No updates in iteration 2, algorithm can terminate early

Negative cycle detection:
  No negative cycle detected

=== Bellman-Ford Results ===
Shortest distances from A: {'A': 0, 'B': 4, 'C': 2, 'D': 9, 'E': 5, 'F': 8, 'G': 6}
Parent pointers: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B',

## Floyd-Warshall Algorithm - All-Pairs Shortest Paths

**Floyd-Warshall Algorithm** finds shortest paths between **all pairs of vertices** in a directed weighted graph. It can handle negative weights but not negative cycles.

**Algorithm Steps:**
1. Initialize distance matrix with edge weights (infinity for no direct edge)
2. Set diagonal elements to 0 (distance from vertex to itself)
3. For each intermediate vertex k:
   - For each pair of vertices (i,j):
     - If path i→k→j is shorter than direct i→j, update distance[i][j]
4. Result: matrix with shortest distances between all vertex pairs

**Time Complexity:** O(V³) - always, regardless of edge count
**Space Complexity:** O(V²) for the distance matrix

**Key Properties:**
- **Dynamic Programming**: Considers all possible intermediate vertices
- **All-pairs solution**: Finds shortest paths between every vertex pair
- **Directional**: Respects edge directions (asymmetric matrix)
- **Negative weight support**: Can handle negative edges (but not negative cycles)

**Use Cases:**
- **Network analysis**: Finding shortest paths between all node pairs
- **Game AI**: Precomputing distances for fast pathfinding queries
- **Transportation**: Route planning with multiple source-destination pairs

In [22]:
def floyd_warshall_directed(graph):
    """
    Floyd-Warshall algorithm for finding shortest paths between all pairs in directed graph
    Returns distance matrix and path reconstruction matrix
    """
    n = len(graph.nodes)
    
    # Initialize distance matrix with adjacency matrix
    distances = [[float('inf') for _ in range(n)] for _ in range(n)]
    next_node = [[None for _ in range(n)] for _ in range(n)]
    
    # Set up initial distances
    for i in range(n):
        distances[i][i] = 0  # Distance from node to itself is 0
    
    # Add existing directed edges
    for u in graph.adj_list:
        u_idx = graph.node_to_index[u]
        for v, weight in graph.adj_list[u]:
            v_idx = graph.node_to_index[v]
            distances[u_idx][v_idx] = weight
            next_node[u_idx][v_idx] = v_idx
    
    print("Starting Floyd-Warshall algorithm for directed graph")
    print("Initial distance matrix:")
    print_matrix_directed(distances, graph.nodes)
    
    # Main Floyd-Warshall algorithm
    for k in range(n):
        k_node = graph.nodes[k]
        print(f"\nIteration {k+1}: Using {k_node} as intermediate vertex")
        
        updates = []
        for i in range(n):
            for j in range(n):
                if distances[i][k] + distances[k][j] < distances[i][j]:
                    old_dist = distances[i][j]
                    distances[i][j] = distances[i][k] + distances[k][j]
                    next_node[i][j] = next_node[i][k]
                    
                    i_node = graph.nodes[i]
                    j_node = graph.nodes[j]
                    updates.append(f"{i_node}→{j_node}: {old_dist} → {distances[i][j]}")
        
        if updates:
            print(f"Updates: {', '.join(updates)}")
        else:
            print("No updates in this iteration")
        
        print("Current distance matrix:")
        print_matrix_directed(distances, graph.nodes)
    
    return distances, next_node

def print_matrix_directed(matrix, nodes):
    """Helper function to print matrix with node labels for directed graph"""
    n = len(nodes)
    print("From\\To", end="")
    for node in nodes:
        print(f"{node:>6}", end="")
    print()
    
    for i in range(n):
        print(f"{nodes[i]:>2}     ", end="")
        for j in range(n):
            val = matrix[i][j]
            if val == float('inf'):
                print("    ∞", end="")
            else:
                print(f"{val:>5}", end="")
        print()

def reconstruct_floyd_path(next_matrix, graph, start, end):
    """Reconstruct path from start to end using Floyd-Warshall next matrix"""
    start_idx = graph.node_to_index[start]
    end_idx = graph.node_to_index[end]
    
    if next_matrix[start_idx][end_idx] is None:
        return []
    
    path = [start]
    current_idx = start_idx
    
    while current_idx != end_idx:
        current_idx = next_matrix[current_idx][end_idx]
        path.append(graph.nodes[current_idx])
    
    return path

# Run Floyd-Warshall algorithm
fw_distances, fw_next = floyd_warshall_directed(graph)

print("\n=== Floyd-Warshall Results ===")
print("Final shortest distances matrix:")
print_matrix_directed(fw_distances, graph.nodes)

print("\nShortest paths between all pairs:")
for i, start in enumerate(graph.nodes):
    for j, end in enumerate(graph.nodes):
        if i != j:
            distance = fw_distances[i][j]
            if distance != float('inf'):
                path = reconstruct_floyd_path(fw_next, graph, start, end)
                print(f"{start} → {end}: {' → '.join(path)} (weight: {distance})")
            else:
                print(f"{start} → {end}: No path exists")

Starting Floyd-Warshall algorithm for directed graph
Initial distance matrix:
From\To     A     B     C     D     E     F     G
 A         0    4    2    ∞    ∞    ∞    ∞
 B         ∞    0    ∞    5    1    ∞    ∞
 C         ∞    3    0    8    ∞   10    ∞
 D         ∞    ∞    ∞    0    2    ∞    6
 E         ∞    ∞    ∞    ∞    0    3    1
 F         ∞    ∞    ∞    ∞    ∞    0    4
 G         7    ∞    ∞    ∞    ∞    ∞    0

Iteration 1: Using A as intermediate vertex
Updates: G→B: inf → 11, G→C: inf → 9
Current distance matrix:
From\To     A     B     C     D     E     F     G
 A         0    4    2    ∞    ∞    ∞    ∞
 B         ∞    0    ∞    5    1    ∞    ∞
 C         ∞    3    0    8    ∞   10    ∞
 D         ∞    ∞    ∞    0    2    ∞    6
 E         ∞    ∞    ∞    ∞    0    3    1
 F         ∞    ∞    ∞    ∞    ∞    0    4
 G         7   11    9    ∞    ∞    ∞    0

Iteration 2: Using B as intermediate vertex
Updates: A→D: inf → 9, A→E: inf → 5, C→E: inf → 4, G→D: inf → 16, G→

## Negative Cycle Detection in Directed Graphs

**Negative cycle detection** is crucial in directed graphs as negative cycles make shortest path problems undefined. Several algorithms can detect them:

### Methods for Negative Cycle Detection:

1. **Bellman-Ford Algorithm**
   - Run V-1 relaxation rounds, then check if any edge can still be relaxed
   - If yes, negative cycle exists
   - **Time Complexity**: O(VE)

2. **Floyd-Warshall Algorithm**  
   - After completion, check if any diagonal element is negative
   - **Time Complexity**: O(V³)

3. **DFS-based Detection**
   - Use DFS with distance tracking to find back edges with negative total weight
   - **Time Complexity**: O(V + E)

**Applications:**
- **Currency arbitrage**: Detecting profitable trading cycles
- **Game theory**: Finding beneficial strategy loops
- **Network optimization**: Identifying impossible routing scenarios

In [23]:
def detect_negative_cycle_comprehensive(graph):
    """
    Comprehensive negative cycle detection using multiple methods
    """
    print("=== Comprehensive Negative Cycle Detection ===")
    
    # Method 1: Using Bellman-Ford from each vertex
    print("\n1. Bellman-Ford Method:")
    negative_cycles_found = []
    
    for start_node in graph.nodes:
        print(f"   Testing from {start_node}:")
        _, _, has_neg_cycle = bellman_ford_directed(graph, start_node)
        if has_neg_cycle:
            negative_cycles_found.append(start_node)
            print(f"   ✗ Negative cycle detected from {start_node}")
        else:
            print(f"   ✓ No negative cycle from {start_node}")
    
    # Method 2: Floyd-Warshall diagonal check
    print("\n2. Floyd-Warshall Method:")
    fw_dist, _ = floyd_warshall_directed(graph)
    fw_negative_cycle = any(fw_dist[i][i] < 0 for i in range(len(graph.nodes)))
    
    if fw_negative_cycle:
        print("   ✗ Negative cycle detected (negative diagonal element)")
        for i, node in enumerate(graph.nodes):
            if fw_dist[i][i] < 0:
                print(f"      {node} has negative self-distance: {fw_dist[i][i]}")
    else:
        print("   ✓ No negative cycle detected")
    
    return len(negative_cycles_found) > 0 or fw_negative_cycle

# Test negative cycle detection on our current graph
has_cycle = detect_negative_cycle_comprehensive(graph)
print(f"\n=== Overall Result ===")
print(f"Negative cycle exists in graph: {has_cycle}")

=== Comprehensive Negative Cycle Detection ===

1. Bellman-Ford Method:
   Testing from A:
Starting Bellman-Ford algorithm from node: A
Directed edge list: [('A', 'B', 4), ('A', 'C', 2), ('B', 'D', 5), ('B', 'E', 1), ('C', 'D', 8), ('C', 'F', 10), ('C', 'B', 3), ('D', 'E', 2), ('D', 'G', 6), ('E', 'F', 3), ('E', 'G', 1), ('F', 'G', 4), ('G', 'A', 7)]
Step by step execution:

Iteration 1:
  Updated A→B: inf → 4 (edge weight: 4)
  Updated A→C: inf → 2 (edge weight: 2)
  Updated B→D: inf → 9 (edge weight: 5)
  Updated B→E: inf → 5 (edge weight: 1)
  Updated C→F: inf → 12 (edge weight: 10)
  Updated D→G: inf → 15 (edge weight: 6)
  Updated E→F: 12 → 8 (edge weight: 3)
  Updated E→G: 15 → 6 (edge weight: 1)
  Current distances: {'A': 0, 'B': 4, 'C': 2, 'D': 9, 'E': 5, 'F': 8, 'G': 6}

Iteration 2:
  No updates in iteration 2, algorithm can terminate early

Negative cycle detection:
  No negative cycle detected
   ✓ No negative cycle from A
   Testing from B:
Starting Bellman-Ford algorithm 

## Algorithm Complexity and Performance Analysis

Let's analyze the time and space complexity of all algorithms for directed weighted graphs:

### Time Complexity Summary

| Algorithm | Best Case | Average Case | Worst Case | Space Complexity |
|-----------|-----------|--------------|------------|------------------|
| **BFS** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **DFS** | O(V + E) | O(V + E) | O(V + E) | O(V) |
| **Dijkstra** | O((V + E) log V) | O((V + E) log V) | O((V + E) log V) | O(V) |
| **Bellman-Ford** | O(VE) | O(VE) | O(VE) | O(V) |
| **Floyd-Warshall** | O(V³) | O(V³) | O(V³) | O(V²) |
| **Chu-Liu/Edmonds'** | O(EV) | O(EV) | O(EV) | O(V + E) |

### Graph Representation Complexity

| Representation | Space | Add Edge | Remove Edge | Check Edge | Get Successors | Get Predecessors |
|----------------|-------|----------|-------------|------------|----------------|-----------------|
| **Adjacency Matrix** | O(V²) | O(1) | O(1) | O(1) | O(V) | O(V) |
| **Adjacency List** | O(V + E) | O(1) | O(out-degree) | O(out-degree) | O(out-degree) | O(V + E) |

### Performance Characteristics by Graph Type

**Dense Directed Graphs (E ≈ V²):**
- **Adjacency Matrix** representation becomes space-efficient
- **Floyd-Warshall** is competitive with repeated Dijkstra
- **Matrix operations** are faster for connectivity queries

**Sparse Directed Graphs (E << V²):**
- **Adjacency List** representation is preferred
- **Dijkstra** from multiple sources often better than Floyd-Warshall
- **Memory usage** is significantly lower

**Directed vs Undirected Considerations:**
- **Edge count**: Directed graphs can have up to V(V-1) edges vs V(V-1)/2 for undirected
- **Reachability**: Not all vertices may be reachable from a given source
- **Cycle detection**: More complex due to directional constraints

## Summary and Comparison Table

Here's a comprehensive comparison of all algorithms and techniques covered for directed weighted graphs:

### Algorithm Classification and Use Cases

| Category | Algorithm | Purpose | Key Strength | Limitation |
|----------|-----------|---------|--------------|------------|
| **Graph Traversal** | BFS | Level-order exploration | Shortest hop count | Only follows outgoing edges |
| | DFS (Preorder) | Depth-first exploration | Memory efficient | May miss reachable nodes |
| | DFS (Postorder) | Finish time tracking | Good for topological sort | Stack overflow risk |
| **Single-Source Shortest Path** | Dijkstra | Non-negative weights | Optimal + efficient | No negative weights |
| | Bellman-Ford | Negative weights allowed | Detects negative cycles | Slower O(VE) |
| **All-Pairs Shortest Path** | Floyd-Warshall | All vertex pairs | Handles negative weights | O(V³) complexity |
| **Spanning Structure** | Chu-Liu/Edmonds' | Minimum spanning arborescence | Directed MST | Complex implementation |

### Time Complexity Comparison

| Algorithm | Time Complexity | Space Complexity | Best For |
|-----------|----------------|------------------|----------|
| **BFS/DFS** | O(V + E) | O(V) | Reachability, traversal |
| **Dijkstra** | O((V + E) log V) | O(V) | Single-source, non-negative |
| **Bellman-Ford** | O(V × E) | O(V) | Negative weights, cycle detection |
| **Floyd-Warshall** | O(V³) | O(V²) | All-pairs, small graphs |
| **Chu-Liu/Edmonds'** | O(EV) | O(V + E) | Directed MST |

### Feature Comparison

| Algorithm | Handles Negative Weights | Detects Negative Cycles | All-Pairs | Respects Direction |
|-----------|------------------------|------------------------|-----------|-------------------|
| **Dijkstra** | No | No | No | Yes |
| **Bellman-Ford** | Yes | Yes | No | Yes |
| **Floyd-Warshall** | Yes | Yes | Yes | Yes |
| **BFS/DFS** | N/A (unweighted) | No | No | Yes |
| **Chu-Liu/Edmonds'** | No | No | No | Yes |

### Algorithm Selection Guide

**For Shortest Paths:**
- **Single source, non-negative weights** → Dijkstra's Algorithm
- **Single source, negative weights** → Bellman-Ford Algorithm
- **All pairs, small graphs** → Floyd-Warshall Algorithm
- **All pairs, large graphs** → Multiple Dijkstra runs
- **Negative cycle detection needed** → Bellman-Ford or Floyd-Warshall

**For Graph Traversal:**
- **Reachability analysis** → BFS (shortest hops) or DFS (memory efficient)
- **Level-by-level exploration** → BFS
- **Topological ordering** → DFS Postorder
- **Cycle detection** → DFS with coloring

**For Spanning Structures:**
- **Minimum spanning arborescence** → Chu-Liu/Edmonds' Algorithm
- **Broadcast tree construction** → MSA algorithms

**Graph Representation Choice:**
- **Dense graphs (E ≈ V²)** → Adjacency Matrix
- **Sparse graphs (E << V²)** → Adjacency List
- **Frequent predecessor queries** → Reverse Adjacency List
- **Memory constrained** → Adjacency List

### Special Considerations for Directed Graphs

| Aspect | Directed Graph Challenge | Solution Approach |
|--------|-------------------------|-------------------|
| **Reachability** | Not all vertices reachable from any source | Check connectivity from multiple sources |
| **Cycles** | Can have complex cycle structures | Use specialized cycle detection algorithms |
| **Symmetry** | Asymmetric relationships (A→B ≠ B→A) | Separate tracking of in/out edges |
| **MST Analog** | No simple MST equivalent | Use Minimum Spanning Arborescence |
| **All-Pairs** | May have many unreachable pairs | Floyd-Warshall handles this naturally |

This completes our comprehensive guide to Directed Weighted Graphs and their fundamental algorithms!

## Summary Table

| Algorithm | Handles Negative Weights | Detects Negative Cycles | All-Pairs | Directed |
|-----------|------------------------|------------------------|-----------|----------|
| Dijkstra | No | No | No | Yes |
| Bellman-Ford | Yes | Yes | No | Yes |
| Floyd-Warshall | Yes | Yes | Yes | Yes |
| Chu-Liu/Edmonds' | No | No | No | Yes |